In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim


In [10]:
def transfer_function(z):
    numerator = -z**2 + 1.9 * z + 0.95
    denominator = z**3 - 0.18 * z**2 + 0.08 * z - 0.08
    return numerator / denominator

In [11]:
def generate_delayed_data(data, delays):
    delayed_data = []
    for delay in delays:
        padded_data = torch.cat([torch.zeros(delay), data[:-delay]])
        delayed_data.append(padded_data)
    return torch.stack(delayed_data, dim=1)


In [12]:
# Example white noise input signal
input_signal = torch.randn(100)
input_delays = [1, 2]
output_delays = [1, 2]

# Generate delayed inputs and outputs
input_data = generate_delayed_data(input_signal, input_delays)
output_data = generate_delayed_data(transfer_function(input_signal), output_delays)


In [ ]:
class StaticEstimator(nn.Module):
    def __init__(self, input_dim, hidden_dim):
        super(StaticEstimator, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, 1)
        self.activation = nn.Sigmoid()

    def forward(self, x):
        x = self.activation(self.fc1(x))
        return self.fc2(x)

# Initialize the model, loss, and optimizer
model = StaticEstimator(input_dim=input_data.shape[1] + output_data.shape[1], hidden_dim=10)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

# Prepare the data
X = torch.cat([input_data, output_data], dim=1)
y = transfer_function(input_signal).view(-1, 1)

# Training loop
num_epochs = 1000
for epoch in range(num_epochs):
    optimizer.zero_grad()
    outputs = model(X)
    loss = criterion(outputs, y)
    loss.backward()
    optimizer.step()

    if epoch % 100 == 0:
        print(f"Epoch {epoch}, Loss: {loss.item()}")
